In [ ]:
import pandas as pd
import numpy as np
import altair as alt
from natsort import natsorted

In [ ]:
file = '../Data/20250508_BARD1scores_update_FILTERED.xlsx' #SGE score file
#file = '/Users/ivan/Downloads/20240828_BARD1_AllScores_noX4.xlsx' #no X4A-K file

In [ ]:
def read_scores(file): #Reads SGE scores
    df = pd.read_excel(file)

    all_columns = ['target','pos_id','D05_R1_lib1','D05_R2_lib1','D05_R3_lib1','D13_R1_lib1','D13_R2_lib1','D13_R3_lib1', 'D05_R1_lib2', 'D05_R2_lib2', 'D05_R3_lib2', 'D13_R1_lib2', 'D13_R2_lib2' ,'D13_R3_lib2'] #List of columns to retain

    df = df[all_columns]
    targets = natsorted(list(set(df['target'].tolist()))) #List of targets

    overlapping_dict = {'BARD1_X1B': 'BARD1_X1A',
                        'BARD1_X3B': 'BARD1_X3A',
                        'BARD1_X4B': 'BARD1_X4A',
                        'BARD1_X4C': 'BARD1_X4B',
                        'BARD1_X4D': 'BARD1_X4C', 
                        'BARD1_X4E': 'BARD1_X4D',
                        'BARD1_X4F': 'BARD1_X4E',
                        'BARD1_X4G': 'BARD1_X4F',
                        'BARD1_X4H': 'BARD1_X4G', 
                        'BARD1_X4I': 'BARD1_X4H', 
                        'BARD1_X4L': 'BARD1_X4K',
                        'BARD1_X5B': 'BARD1_X5A', 
                        'BARD1_X6B': 'BARD1_X6A', 
                        'BARD1_X6C': 'BARD1_X6B',
                        'BARD1_X7B': 'BARD1_X7A',
                        'BARD1_X8B': 'BARD1_X8A',
                        'BARD1_X9B': 'BARD1_X9A', 
                        'BARD1_X10B': 'BARD1_X10A', 
                        'BARD1_X11B': 'BARD1_X11A', 
                        'BARD1_X11C': 'BARD1_X11B', 
                        'BARD1_X11D': 'BARD1_X11C'
                       } #Dicitonary for overlapping regions second region: first overlapping region

    all_dfs = [] #List of target-by-target dfs
    overlapping_libraries = list(overlapping_dict.keys()) #Gets keys of overlapping libraries dictionary
    
    for target in targets: #Iterates through all targets and pulls relevant counts
        target_df = df.loc[df['target'].isin([target])] #Gets non-overlapping counts

        target_df = target_df[['target', 'pos_id','D05_R1_lib1','D05_R2_lib1','D05_R3_lib1','D13_R1_lib1','D13_R2_lib1','D13_R3_lib1']] #Pulls these columns only 
        
        if target in overlapping_libraries: #For targets with overlaps, gets overlapping counts in the "lib2" columns
            overlap_df = df.loc[(df['target'].isin([overlapping_dict[target]])) & (df['D05_R1_lib2'].notna())] #Gets overlapping counts
            overlap_df = overlap_df.copy() #Copy to avoid set-copy error
            overlap_df['target'] = target #Sets target name to correct target name
            overlap_df = overlap_df[['target','pos_id', 'D05_R1_lib2', 'D05_R2_lib2', 'D05_R3_lib2', 'D13_R1_lib2', 'D13_R2_lib2' ,'D13_R3_lib2']] #Extracts counts
            overlap_df = overlap_df.rename(columns = {'D05_R1_lib2': 'D05_R1_lib1',
                                                      'D05_R2_lib2': 'D05_R2_lib1',
                                                      'D05_R3_lib2': 'D05_R3_lib1',
                                                      'D13_R1_lib2': 'D13_R1_lib1',
                                                      'D13_R2_lib2': 'D13_R2_lib1',
                                                      'D13_R3_lib2': 'D13_R3_lib1'
                                                     }
                                          ) #Renames counts
            

            final_df = pd.concat([target_df, overlap_df]) #Non-overlapping and overlapping counts concatenated
            all_dfs.append(final_df) #Dataframe appended to list
            
        else: #For regions without overlaps with a preceding library
            final_df = target_df #Dataframe unchanged
            all_dfs.append(final_df) #Appended to list

    df = pd.concat(all_dfs) #All dataframes concatenated 
    
    df = df.rename(
        columns = {
            'D05_R1_lib1': 'D05 R1', 
            'D05_R2_lib1': 'D05 R2', 
            'D05_R3_lib1': 'D05 R3', 
            'D13_R1_lib1': 'D13 R1',
            'D13_R2_lib1': 'D13 R2', 
            'D13_R3_lib1': 'D13 R3',
        }
    ) #Renames columns to be consistent with downstream code
    
    df['target'] = df['target'].transform(lambda x: x[7::]) #Rewrites targets column to contain target name only

    #df.to_excel('/Users/ivan/Desktop/test_excel_outputs/20250721_BARD1_RCorrCountsCheck.xlsx', index = None) #Debuggig output file
    
    return df

In [ ]:
def group_targets(df): #Groups targets for correlation testing
    
    grouped = df.groupby('target') #Creates groupby objects for each SGE region
    r_lists = [] #list that will hold lists that have correlation
    combos = [('D05 R1','D05 R2'),('D05 R1','D05 R3'),('D05 R2','D05 R3'),
              ('D13 R1','D13 R2'),('D13 R1','D13 R3'),('D13 R2','D13 R3')
             ] #pairwise arguments for correlation testing
    
    for group_name, group_df in grouped: #iterates through groupby objects
        group_df = group_df[['D05 R1', 'D05 R2', 'D05 R3', 
           'D13 R1', 'D13 R2', 'D13 R3']] #pulls out count columns only
        
        for elem in combos: #iterates through each combination of replicates for r-testing
            output = [] #output list
            col1, col2 = elem #unpacks tuple that is combination
            output.append(group_name) #adds SGE target name
            output.append(col1 + ' vs ' + col2) #adds which replicates were tested
            output.append(compute_r(group_df,col1,col2)) #appends r
            r_lists.append(output) #appends final list to output list

    #print(r_lists)
    return r_lists
    

In [ ]:
def compute_r(group,col1,col2): #does the correlation math
    return group[col1].corr(group[col2])

In [ ]:
def parse_r_lists(r_list): #Parses through grouped lists and create Pearson R values
    
    target = [] #list to hold SGE target names
    test_type = [] #list to hold the replicates compared
    r = [] #list to hold r values
    
    for elem in r_list: #iterates through each item in list and appends respective information
        target.append(elem[0])
        test_type.append(elem[1])
        r.append(round(elem[2],3))

    data = {
            'Targets' : target,
            'Tests' : test_type,
            'r_correlation': r
    } #Dictionary to build dataframe
    
    to_map = pd.DataFrame(data) #Final dataframe with all R-values

    to_map['exon'] = to_map['Targets'].str.extract(r'^(\d+\.?\d*)').astype(str)

    agg_df = to_map.groupby('Targets').agg({'r_correlation': 'median',
                                            'exon': 'first'
                                           }
                                          ).reset_index()

    #agg_df.to_excel('/Users/ivan/Desktop/test_excel_outputs/20250721_AggMedianRCorrelation.xlsx', index = None)
    
    return to_map

In [ ]:
def heatmap(pivoted):

    targets = set(pivoted['Targets'].tolist())
    targets = natsorted(targets)
    
    graph = alt.Chart(pivoted, title = alt.TitleParams(text = 'Correlation of Replicates', fontSize = 32)).mark_rect().encode(
                x = alt.X('Tests:N', axis = alt.Axis(title = '', titleFontSize = 28, labelFontSize = 24, labelLimit = 300, labelAngle = 45)),
                y = alt.Y('Targets', axis = alt.Axis(title = 'SGE Target Region', titleFontSize = 28, labelFontSize = 24), sort = targets),
                color = alt.Color('r_correlation:Q', scale = alt.Scale(domain = [.2, 1]), legend = alt.Legend(title = "Pearson's r", titleFontSize = 24,labelFontSize = 22)),
                tooltip = [alt.Tooltip('r_correlation', title = "Pearson's r: ")]
    ).properties(
        width = 600,
        height = 900
    )

    #graph.save('/Users/ivan/Desktop/BARD1_draft_figs/fig_1b.png', ppi = 500)
    graph.display()


In [ ]:
def boxplot(df):

    df_all = df.copy()
    df_all['Domain'] = 'All Data'
    df_all['Structure'] = 'All Data'

    
    df['Domain'] = None
    df.loc[df['Targets'].isin(['1B, 2, 3A, 3B']), 'Domain'] = 'RING'
    df.loc[df['Targets'].isin(['4A', '4B', '4C', '4D', '4E', '4F', '4G', '4H', '4I', '4J', '4K']),'Domain'] = 'IDR'
    df.loc[df['Targets'].isin(['4L', '5A', '5B', '6A', '6B', '7A', '7B']), 'Domain'] = 'ARD'
    df.loc[df['Targets'].isin(['8A','8B', '9A', '9B', '10A', '10B', '11A', '11B', '11C', '11D']), 'Domain'] ='BRCT'

    df['Structure'] = None
    df.loc[~(df['Targets'].isin(['4A', '4B', '4C', '4D', '4E', '4F', '4G', '4H', '4I', '4J', '4K'])),'Structure'] = 'Structured\nDomains'
    df.loc[df['Targets'].isin(['4A', '4B', '4C', '4D', '4E', '4F', '4G', '4H', '4I', '4J', '4K']),'Structure'] = 'Disordered\nDomain'

    df = df.dropna(subset = ['Structure'])

    df = pd.concat([df_all, df])

    df['Day'] = None
    df.loc[df['Tests'].str.contains('D05'), 'Day'] = 'Day 5'
    df.loc[df['Tests'].str.contains('D13'), 'Day'] = 'Day 13'

    d5_df = df.loc[df['Day'].isin(['Day 5'])]
    d13_df = df.loc[df['Day'].isin(['Day 13'])]

    palette = [
    '#006616', # dark green,
    '#81B4C7', # dusty blue
    '#ffcd3a', # yellow
    ]

    variant_types = [
        'All Data',
        'Structured\nDomains',  
        'Disordered\Domain'
    ]
    

    plot = alt.Chart(df).mark_boxplot(
        size = 75
    ).encode(
        y = alt.Y('r_correlation', 
                  scale = alt.Scale(
                    domain = [0.45, 1.02]
                    ),
                  axis = alt.Axis(title = "",
                                  labelFontSize = 16,
                                  values = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
                                 )
        ),
        x = alt.X('Structure',
                 axis = alt.Axis(labelAngle = 0,
                                title = ''),
                  sort = ['All Data', 'Structured\nDomains', 'Disordered\nDomain']
                 ),
        color = alt.Color('Structure',
                        scale = alt.Scale(range = palette,
                                          scheme = variant_types
                                         ),
                         legend = None)
    ).configure_view(
        stroke = None
    ).configure_axis(
        grid = False
    ).properties(
        width = 300,
        height = 500
    ).interactive()

    plot.display()

    days_df = [d5_df, d13_df]
    day = ["Day 5", "Day 13"]
    save_day = ['d5_boxplot', 'd13_plot']
    
    days_plots = []
    for i, df in enumerate(days_df):

        title_str = day[i]
        save_str = '/Users/ivan/Desktop/BARD1_draft_figs/fig_1b_' + save_day[i] + '.png'

        # Create a copy and rename the Structure column with a unique suffix
        df_copy = df.copy()
        df_copy[f'Structure_{i}'] = df_copy['Structure']
        
        plot = alt.Chart(df_copy).mark_boxplot(
            size = 75
        ).encode(
            y = alt.Y('r_correlation', 
                      scale = alt.Scale(
                        domain = [0.45, 1.02]
                        ),
                      axis = alt.Axis(title = "",
                                      labelFontSize = 18,
                                      values = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
                                     )
            ),
            x = alt.X(f'Structure_{i}',  # Use the renamed column
                     axis = alt.Axis(labels = False,
                                    title = ''),
                      sort = ['All Data', 'Structured\nDomains', 'Disordered\nDomain']
                     ),
            color = alt.Color(f'Structure_{i}', # Use the renamed column
                              scale = alt.Scale(range = palette,
                                          scheme = variant_types
                                         ),
                              legend = alt.Legend(title = '',
                                                  labelFontSize = 18
                                                 )
                            )
        ).properties(
            width = 300,
            height = 500,
            title = title_str
        ).configure_view(
            stroke = None
        ).configure_axis(
            grid = False
        ).interactive()

        plot.display()
        #plot.save(save_str, ppi = 500)
    

In [ ]:
def main():
    data = read_scores(file) 
    r_lists = group_targets(data)
    test = parse_r_lists(r_lists)

    heatmap(test)
    boxplot(test)

In [ ]:
main()